### Milestone 1: Web Scrapping
WQD180102 Ng Wei Xin <br>
WQD180104 Tan Bing Shien

In [2]:
# pip install requests
# pip install beautifulsoup4

import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

# using requests to fetch pages
# Note: Python Selenium library can be used for javascript pages (by imitating browser behaviour)

# bs4: Html-Parser (or XML), to navigate through html documents

In [3]:
# Get url content using python-requests
page = requests.get('https://www.thestar.com.my/news/latest')
contents = page.content

# Parse html document in bs4
soup = BeautifulSoup(contents, 'html.parser')
print(soup.prettify())

<!--default_base.blade.php-->
<!DOCTYPE html>
<html lang="en">
 <head>
  <title>
   Latest - News | The Star Online
  </title>
  <link href="https://cdn.thestar.com.my/Themes/img/favicon.ico" rel="icon" type="image/png"/>
  <!--START: common.blade.php-->
  <meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
  <script type="text/javascript">
   (window.NREUM||(NREUM={})).loader_config={licenseKey:"ef8b08f3e1",applicationID:"379274163"};window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var i=n[t]={exports:{}};e[t][0].call(i.exports,function(n){var i=e[t][1][n];return r(i||n)},i,i.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<t.length;i++)r(t[i]);return r}({1:[function(e,n,t){function r(){}function i(e,n,t){return function(){return o(e,[u.now()].concat(f(arguments)),n?null:this,t),n?void 0:this}}var o=e("handle"),a=e(4),f=e(5),c=e("ee").get("tracer"),u=e("loader"),s=NREUM;"undefined"==typeof wi

In [4]:
# Create empty dict
# Note: using news-id as identifier -> prevent duplication
myList = {}

# Use class "timeline-content" to identify news section
for tag in soup.find_all(class_= "timeline-content", limit=15):

    # Further navigate to "h2" tag
    for subtag in tag.find('h2'):
        
        # Get String (news headline), trim / strip linebreaks
        strNews = "".join(line.strip() for line in subtag.string.split("\n"))
        idNews = "".join(line.strip() for line in subtag['data-content-id'].split("\n"))
        
        # Crawl sub page, get Date / Timestamp
        newsHref = subtag['href']
        subPage = requests.get(newsHref)
        subContent = subPage.content
        subSoup = BeautifulSoup(subContent, 'html.parser')
        timeTag = "".join(line.strip() for line in subSoup.find('time', {'class': 'timestamp'}).string.split("\n"))
        dateTag = "".join(line.strip() for line in subSoup.find('p', {'class': 'date'}).string.split("\n"))
        
        # (print) News-ID: News-String
        print("News-ID ("+idNews+"):")
        print(strNews)
        print(newsHref)
        print(dateTag)
        print(timeTag)
        print()
        
        # Add entry to list 
        myList[subtag['data-content-id']]=strNews

News-ID (230548):
North Sumatra urged to export more swallow's nest products
https://www.thestar.com.my/news/regional/2020/03/12/north-sumatra-urged-to-export-more-swallow039s-nest-products
Thursday, 12 Mar 2020
6:17 PM MYT

News-ID (230485):
Virtual reality game lets Dutch patients 'shoot' their pain
https://www.thestar.com.my/tech/tech-news/2020/03/12/virtual-reality-game-lets-dutch-patients-039shoot039-their-pain
Thursday, 12 Mar 2020
6:15 PM MYT

News-ID (230538):
Malindo Air offers unpaid leave, cuts flight capacity by half
https://www.thestar.com.my/business/business-news/2020/03/12/malindo-air-offers-unpaid-leave-cuts-flight-capacity-by-half
Thursday, 12 Mar 2020
6:14 PM MYT

News-ID (230054):
Fear of virus-tainted dollars opens new front in war on cash
https://www.thestar.com.my/tech/tech-news/2020/03/12/fear-of-virus-tainted-dollars-opens-new-front-in-war-on-cash
Thursday, 12 Mar 2020
6:10 PM MYT

News-ID (230500):
Priyanka Chopra and Nick Jonas celebrate Holi together for the

In [6]:
def recrawl(objDF):
    page = requests.get('https://www.thestar.com.my/news/latest')
    contents = page.content

    soup = BeautifulSoup(contents, 'html.parser')

    for tag in soup.find_all(class_= "timeline-content", limit = 15):
        for subtag in tag.find('h2'):
            
            # News-ID
            idNews = "".join(line.strip() for line in subtag['data-content-id'].split("\n"))
                        
            # Check for duplication, proceed only for new item
            # else skip to save computation
            if not df['News_ID'].str.contains(idNews).any():
                
                # Headline
                strNews = "".join(line.strip() for line in subtag.string.split("\n"))
                
                # Category
                catNews = "".join(line.strip() for line in subtag['data-content-category'].split("\n"))
                
                # Timestamp (spider on respective news page)
                newsHref = subtag['href']
                subPage = requests.get(newsHref)
                subContent = subPage.content
                subSoup = BeautifulSoup(subContent, 'html.parser')
                dateNews = "".join(line.strip() for line in subSoup.find(class_="date").string.split("\n"))
                timeNews = "".join(line.strip() for line in subSoup.find(class_="timestamp").string.split("\n"))
                
                # Add info to list
                objDF = objDF.append(pd.Series([idNews, dateNews, timeNews, catNews, strNews, newsHref], index=objDF.columns), ignore_index=True)

    return objDF

In [8]:
# Create Empty DataFrame:
df= pd.DataFrame(columns=["News_ID", "Date", "Timestamp", "Category", "Headline", "URL"])

# Endless-Loop
# while True: 
    
i = 1 # Test: using 3 counts 
while i>0:
    
    df = recrawl(df)
    print(df)
    
    # Export dataframe to csv file
    df.to_csv(r'C:\Users\FORGE-15 I7\OneDrive - AsiaPay Limited\Sem 3\WQD7005 DATA MINING\dataset.csv', index = False)
#     df.to_csv(r'C:\Users\ngwei\Desktop\dataset.csv', index = False)
    
    i=i-1
    #time.sleep(300) # Re-Crawl every 5 minutes


   News_ID                   Date    Timestamp                 Category  \
0   230569  Thursday, 12 Mar 2020  6:21 PM MYT              News/Nation   
1   230548  Thursday, 12 Mar 2020  6:17 PM MYT            News/Regional   
2   230485  Thursday, 12 Mar 2020  6:15 PM MYT           Tech/Tech News   
3   230538  Thursday, 12 Mar 2020  6:14 PM MYT   Business/Business News   
4   230054  Thursday, 12 Mar 2020  6:10 PM MYT           Tech/Tech News   
5   230500  Thursday, 12 Mar 2020  6:00 PM MYT  Lifestyle/Entertainment   
6   230502  Thursday, 12 Mar 2020  5:59 PM MYT            News/Regional   
7   230499  Thursday, 12 Mar 2020  5:56 PM MYT              News/Nation   
8   230062  Thursday, 12 Mar 2020  5:55 PM MYT           Tech/Tech News   
9   230451  Thursday, 12 Mar 2020  5:45 PM MYT           Tech/Tech News   
10  230482  Thursday, 12 Mar 2020  5:40 PM MYT   Business/Business News   
11  230449  Thursday, 12 Mar 2020  5:40 PM MYT           Tech/Tech News   
12  230457  Thursday, 12 

In [ ]:
# save to csv
# df.to_csv(r'C:\Users\ngwei\Desktop\dataset.csv', index = False)


# # Save scraped content into a txt file

# file1 = open("tmpcrawl.txt","w") 

# # file1.write("Hello \n") 
# # file1.write(str(soup.prettify())) 
# file1.write(str(myList)) 

# file1.close() 
